В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [3]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 43.0 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [ ]:
info = api.info()
info

In [31]:
w2v_model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [8]:
def ten_words (word):
  similar_words = w2v_model.most_similar(word, topn=10)
  return similar_words

In [11]:
ten_words('russia')

[('ukraine', 0.8513927459716797),
 ('russian', 0.766777753829956),
 ('moscow', 0.7559292912483215),
 ('germany', 0.7488042116165161),
 ('europe', 0.7311649322509766),
 ('canada', 0.7134655117988586),
 ('sweden', 0.7122511267662048),
 ('italy', 0.7001978754997253),
 ('iran', 0.6923622488975525),
 ('greece', 0.6922304034233093)]

3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [14]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [12]:
model = Word2Vec(
    sentences = cooking_sentences,
    vector_size = 50,
    window = 3,
    min_count = 1,
    workers=2,
    sg = 1
)

In [16]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [ ]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

In [16]:
try:
  similar = model.wv.most_similar('духовка', topn=3)
  similar_1 = model.wv.most_similar('овощи', topn=3)
  print("Слова похожие на духовка:")
  for word, score in similar:
    print(f'{word}:{score:.4f}')

  print('-'*10)
  print("Слова похожие на овощи:")
  for word, score in similar_1:
    print(f'{word}:{score:.4f}')
except KeyError:
  print("Слово не найдено")

Слова похожие на духовка:
ингредиенты:0.3199
десерт:0.3064
холодильник:0.2705
----------
Слова похожие на овощи:
мариновать:0.2716
хлеб:0.2691
гриль:0.2546


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [18]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [26]:
def sim_words(target_word):
  sim = ft_model.wv.most_similar(target_word, topn=3)
  print(f'Похожие слова на {target_word}')
  for word, score in sim:
    print(f'{word}:{score:.4f}')
  print("-"*10)

In [27]:
sim_words('варить')
sim_words('духовка')
sim_words('овощи')

Похожие слова на варить
жарить:0.5353
парить:0.4805
месить:0.3541
----------
Похожие слова на духовка
взбивать:0.4565
лимон:0.3561
салат:0.3050
----------
Похожие слова на овощи
жарить:0.2960
фольга:0.2574
морковь:0.2297
----------


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [28]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('neural')


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'neural'
  Word2Vec: слово не найдено
  FastText: ['мука', 'травы']


In [30]:
compare_models('мка')
compare_models('удховк')


Сравнение для слова: 'мка'
  Word2Vec: слово не найдено
  FastText: ['яблоки', 'кипятить']

Сравнение для слова: 'удховк'
  Word2Vec: слово не найдено
  FastText: ['вино', 'духовка']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [4]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [5]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [9]:
print(f"Количество документов: {doc_model.dv.key_to_index}")

Количество документов: {'doc_0': 0, 'doc_1': 1, 'doc_2': 2, 'doc_3': 3, 'doc_4': 4}


In [6]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [7]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [10]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [12]:
print(doc_model.dv.most_similar('doc_1', topn=1))

[('doc_0', 0.2735169529914856)]


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [24]:
model_10 = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)

In [25]:
model_50 = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

In [26]:
model_100 = FastText(
    sentences=cooking_sentences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2
)

In [22]:
def sim_words(target_word, model):
  sim = model.wv.most_similar(target_word, topn=5)
  print(f'Модель: {model}')
  print(f'Похожие слова на {target_word}')
  for word, score in sim:
    print(f'{word}:{score:.4f}')
  print("-"*10)

In [28]:
sim_words('питание', model_10)
sim_words('питание', model_50)
sim_words('питание', model_100)

Модель: FastText<vocab=65, vector_size=10, alpha=0.025>
Похожие слова на питание
брокколи:0.6192
чашка:0.5894
овощи:0.5705
фольга:0.5430
гриль:0.4628
----------
Модель: FastText<vocab=65, vector_size=50, alpha=0.025>
Похожие слова на питание
тост:0.4360
ингредиенты:0.3002
десерт:0.2701
тушить:0.2517
пирог:0.2273
----------
Модель: FastText<vocab=65, vector_size=100, alpha=0.025>
Похожие слова на питание
гриль:0.2414
взбивать:0.2202
соль:0.2177
уголь:0.1948
смешивать:0.1739
----------


In [29]:
sim_words('лимон', model_10)
sim_words('лимон', model_50)
sim_words('лимон', model_100)

Модель: FastText<vocab=65, vector_size=10, alpha=0.025>
Похожие слова на лимон
овощи:0.6758
говядина:0.6161
торт:0.5697
яичница:0.5156
холодильник:0.4705
----------
Модель: FastText<vocab=65, vector_size=50, alpha=0.025>
Похожие слова на лимон
духовка:0.3561
тост:0.2885
месить:0.2392
уголь:0.2231
начинка:0.2039
----------
Модель: FastText<vocab=65, vector_size=100, alpha=0.025>
Похожие слова на лимон
мясо:0.2327
тост:0.2165
кипятить:0.2032
бекон:0.1570
смешивать:0.1366
----------


In [30]:
sim_words('кипятить', model_10)
sim_words('кипятить', model_50)
sim_words('кипятить', model_100)

Модель: FastText<vocab=65, vector_size=10, alpha=0.025>
Похожие слова на кипятить
дрожжи:0.5258
вода:0.4059
завтрак:0.3731
картофель:0.3508
тесто:0.3356
----------
Модель: FastText<vocab=65, vector_size=50, alpha=0.025>
Похожие слова на кипятить
огурцы:0.3144
торт:0.2939
картофель:0.2861
жарить:0.2780
здоровое:0.2505
----------
Модель: FastText<vocab=65, vector_size=100, alpha=0.025>
Похожие слова на кипятить
тушить:0.2590
сахар:0.2402
готовить:0.2147
лимон:0.2032
мясо:0.1964
----------
